In [58]:
import re, json, glob
import pandas as pd
import mysql.connector

# load easy & normal -> join

In [109]:
# easy
easy = pd.read_json('nhkwebeasy.json')

# nhkweb20XX.json
jsons = sorted(glob.glob('nhkweb20*.json'))
norm = pd.read_json(jsons[0])
for j in jsons[1:]:
    norm = pd.concat([norm, pd.read_json(j)])
    
# joined table
df = pd.merge(easy,norm, how='inner', on="id")

In [64]:
# join list into string
df = df.sort_values('datePublished')
df['genre'] = df.genre.apply(lambda x:'_'.join(x))
df['keywords'] = df.keywords.apply(lambda x:'_'.join(x))

In [66]:
# select columns
df = df[['id','title_easy_ruby','article_easy_ruby','title','article','genre','keywords','datePublished']]
df.head()

,id,title_easy_ruby,article_easy_ruby,title,article,genre,keywords,datePublished
5179,k10013902141000,<ruby>小笠原諸島<rt>おがさわらしょとう</rt></ruby>から<ruby>外<...,<ruby>世界遺産<rt>せかいいさん</rt></ruby>の<ruby>小笠原諸島<r...,“原発政策ころころ変わる”,野田総理大臣は、千葉県四街道市で街頭演説し、日本未来の党や日本維新の会が掲げる原子力政策につ...,政治,総理大臣_閣議決定_エネルギー政策,2012-12-02T18:09
5203,k10014002951000,オーストラリア <ruby>店<rt>みせ</rt></ruby>で<ruby>男<rt>お...,"<span class=""colorL"">オーストラリア</span>の<span clas...",六本木襲撃事件 １５人に逮捕状,ことし９月、東京・六本木のクラブで、客の男性が男らに金属バットのようなもので殴られて死亡した...,社会,関東連合_六本木_逮捕状,2012-12-06T16:13
5218,k10014085401000,<ruby>新<rt>あたら</rt></ruby>しいロケット「Ｈ３」を<ruby>来年度...,"<span class=""colorC""><ruby>文部科学省<rt>もんぶかがくしょう<...",専門家“ミサイル内部の機器不具合か”,北朝鮮が発射を予告した事実上の弾道ミサイルを、１０日、発射しなかったことについて、海上自衛隊...,社会,チェック_専門家_キム・ジョンイル,2012-12-10T15:51
5302,k10014344461000,<ruby>日本<rt>にっぽん</rt></ruby>と<ruby>中国<rt>ちゅうごく...,"<span class=""colorC""><ruby>埼玉県<rt>さいたまけん</rt><...",新華社“島巡る争いが試練”,中国国営の新華社通信は、ことしの中国外交について、日本政府による沖縄県の尖閣諸島の国有化や南...,国際,評論_新華社_通信,2012-12-21T12:01
5335,k10014441691000,たくさんの<ruby>情報<rt>じょうほう</rt></ruby>を<ruby>集<rt>...,"<span class=""colorC""><ruby>総務省<rt>そうむしょう</rt><...",鉄板直撃２人死亡 会社を捜索,２５日、広島県東広島市の国道を走っていた大型トレーラーの荷台から崩れ落ちた鉄板が乗用車を直撃...,社会,,2012-12-26T13:49


In [53]:
# save as json
data = []
for i, row in df.iterrows():
    data.append(dict(zip(df2.columns, row)))
    
with open('joined_nhk.json','w',encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# load joined data

In [123]:
df = pd.read_json('joined_nhk.json')
df.head()

,id,title_easy_ruby,article_easy_ruby,title,article,genre,keywords,datePublished
0,k10013902141000,<ruby>小笠原諸島<rt>おがさわらしょとう</rt></ruby>から<ruby>外<...,<ruby>世界遺産<rt>せかいいさん</rt></ruby>の<ruby>小笠原諸島<r...,“原発政策ころころ変わる”,野田総理大臣は、千葉県四街道市で街頭演説し、日本未来の党や日本維新の会が掲げる原子力政策につ...,政治,総理大臣_閣議決定_エネルギー政策,2012-12-02T18:09
1,k10014002951000,オーストラリア <ruby>店<rt>みせ</rt></ruby>で<ruby>男<rt>お...,"<span class=""colorL"">オーストラリア</span>の<span clas...",六本木襲撃事件 １５人に逮捕状,ことし９月、東京・六本木のクラブで、客の男性が男らに金属バットのようなもので殴られて死亡した...,社会,関東連合_六本木_逮捕状,2012-12-06T16:13
2,k10014441691000,たくさんの<ruby>情報<rt>じょうほう</rt></ruby>を<ruby>集<rt>...,"<span class=""colorC""><ruby>総務省<rt>そうむしょう</rt><...",鉄板直撃２人死亡 会社を捜索,２５日、広島県東広島市の国道を走っていた大型トレーラーの荷台から崩れ落ちた鉄板が乗用車を直撃...,社会,,2012-12-26T13:49
3,k10014577151000,<ruby>有吉<rt>ありよし</rt></ruby><ruby>佐和子<rt>さわこ</...,"<span class=""colorN""><ruby>有吉<rt>ありよし</rt></ru...",公明・山口代表“日中関係立て直しを”,公明党の山口代表は党の仕事始めの会合で、「日中関係が良好になることが世界の繁栄と安定につなが...,政治,,2013-01-04T11:54
4,k10014638031000,<ruby>料理<rt>りょうり</rt></ruby>を<ruby>手伝<rt>てつだ</...,"<span class=""colorC""><ruby>農林中央金庫<rt>のうりんちゅうおう...",経済団体 “日本再生元年に”,経団連など３つの経済団体による新年のパーティーが開かれ、経済同友会の長谷川代表幹事は、２０１...,ビジネス,,2013-01-07T18:47


In [127]:
# check duplicate
df.id.value_counts().head(10)

k10010815341000    1
k10010270861000    1
k10010860241000    1
k10010913181000    1
k10015182271000    1
k10011021111000    1
k10012073121000    1
k10014940341000    1
k10010467781000    1
k10010843331000    1
Name: id, dtype: int64

In [128]:
df[df.id=='k10014610131000']

,id,title_easy_ruby,article_easy_ruby,title,article,genre,keywords,datePublished
674,k10014610131000,「テロに<ruby>負<rt>ま</rt></ruby>けない」フランスで３７０<ruby>...,"<ruby>７日<rt>なのか</rt></ruby>、<span class=""color...",テロに抗議のデモ行進 １６０万人以上参加,フランスで１７人が死亡した一連のテロ事件を受けて、犠牲者を追悼し、テロに抗議する大規模なデモ...,国際,イスラム教徒_メルケル_デモ行進,2015-01-12T04:42


In [134]:
df.keywords.apply(len).sort_values()

2856      0
2753      0
2754      0
4898      0
2756      0
       ... 
5487     85
5469     89
5489     94
5418    100
5471    121
Name: keywords, Length: 5714, dtype: int64

# SQL

In [136]:
db_settings = {
    "host": '127.0.0.1',
    "database": 'nhkweb',
    "user": 'root',
    "password":"nozomi8931",
    "port":'3306'
}
con = mysql.connector.connect(**db_settings)
cur = con.cursor()

In [130]:
con.close()

In [92]:
for row in cur.fetchall():
    print(row)

('nhk_joined',)


In [137]:
for i, row in df.iterrows():
    cur.execute(f"INSERT INTO nhk_joined VALUES {tuple(row)}")
con.commit()

In [85]:
df.columns

Index(['id', 'title_easy_ruby', 'article_easy_ruby', 'title', 'article',
       'genre', 'keywords', 'datePublished'],
      dtype='object')

In [ ]:
df